In [ ]:
%matplotlib inline
import os
import pathlib

import matplotlib
import matplotlib.pyplot as plt

import numpy as np
import tensorflow as tf
import datetime

import keras
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, Activation, Reshape, GlobalAveragePooling2D
from keras.models import load_model
from keras.backend.tensorflow_backend import set_session
from keras.callbacks import TensorBoard

import skimage
from skimage import io, color, transform

In [ ]:
# def set_gpu_mem_alloc(mem_use):
#     avail  = 4041
#     percent = mem_use / avail
#     config = tf.ConfigProto()
#     config.gpu_options.per_process_gpu_memory_fraction = percent
#     config.gpu_options.visible_device_list = "0"
#     set_session(tf.Session(config=config))
    
# set_gpu_mem_alloc(2000)

In [ ]:
batch_size = 4
h,w,d = 224,224,3
epochs = 500
date_str = datetime.datetime.now().strftime("%d%m%y_%H%M")

In [ ]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1./255, rotation_range=13.75, 
    horizontal_flip=True, vertical_flip=True,
    shear_range=0.2,
    width_shift_range=0.1, height_shift_range=0.1,
    zoom_range=0.5                            
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        'dataset/animal/train', 
        target_size=(h, w), 
        batch_size=batch_size,
        class_mode='categorical') 
# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        'dataset/animal/valid',
        target_size=(h, w),
        batch_size=batch_size,
        class_mode='categorical')

In [ ]:
x,y = train_generator.next()

In [ ]:
num_class = len(train_generator.class_indices)
train_step = len(train_generator.classes)
valid_step = len(validation_generator.classes)

print('NUM Classes => ', num_class)
print('Train Step  => ',  train_step)
print('Valid Step  => ',  valid_step)

In [ ]:
ksize = (3,3)
psize = (2,2)

input_shape = (h,w,d)

model = Sequential()
model.add(Conv2D(32, kernel_size=ksize, activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=psize))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_class, activation='softmax'))
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc','mse'])

In [ ]:
# untuk liat log history
tb1_path = 'log/'+date_str
pathlib.Path(tb1_path).mkdir(parents=True, exist_ok=True) 
#training
tensorboard = TensorBoard( log_dir=tb1_path, histogram_freq=0, 
                            write_graph=True, write_images=True)

In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch=train_step // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=valid_step // batch_size,
    callbacks=[tensorboard],
    verbose=2
)

In [ ]:
model_path = './saved/svgg'+date_str+'_'+str(epochs)+'x.h5'
model.save(model_path)

In [ ]:
x, y = validation_generator.next()
img = x[0] * 255
img = img.astype("uint8")
plt.imshow(img, cmap = 'gray')

for yy in y:
    print(np.argmax(yy)+1)

In [ ]:
pred = model.predict(x)
for pr in pred:
    print(np.argmax(pr)+1)